# SINGLE IV CURVE ANALYSIS

This script allows you to analyzed a SINGLE IV CURVE (with bias) step by step, just by specifing the file path.
ATTENTION: the input file must be root file containing a TTree with two Branches: BIAS (one value for each endpoint), TRIM and CURRENT (for new data from 14/03/24).

In [ ]:
import IPython
from os import chdir, listdir, path
from uproot import open as op
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
your_path = "/".join(IPython.extract_module_locals()[1]["__vsc_ipynb_file__"].split("/")[:-1])
# print(your_path)
chdir(your_path)
from iv_utl import *

INPUT:

In [ ]:
your_path = '/eos/experiment/neutplatform/protodune/experiments/ProtoDUNE-II/PDS_Commissioning/ivcurves/Apr-19-2024-run00/'
endpoints = '105'
mychannel = 0

all_folders = sorted([folder for folder in listdir(f'{your_path}') if path.isdir(f'{your_path}/{folder}')])
folder = [item for item in all_folders if item.endswith(endpoints)][0]; print(f'Analysing endpoint {endpoints}')
filename = [root_file for root_file in listdir(f'{your_path}{folder}') if root_file.endswith(f'ch_{mychannel}.root')][0]
print(f'Analysing {your_path}{folder}{filename}')

Information about the IV curve: 

In [ ]:
plt_name = (filename.split('/')[-1]).split('.')[0]
endpoint = (plt_name.split('_')[-1]).split('.')[0]
apa = (plt_name.replace('apa_','')).split('_')[0]
chn = int(plt_name.split('ch_')[-1])
afe = chn//8

Read the input file as dataframe:

In [ ]:
root_file = op(f'{your_path}{folder}/{filename}')
directory, trees = get_tree_info(root_file)
print(f'In this configuration you have TDirectories: {directory}, TTrees: {trees}\n')
array_dict = {}
for tree in trees:
    tree = tree.split(";1")[0]
    path_to_var = f'{directory.split(";1")[0]}/{tree}'
    for var in root_file[path_to_var].keys():
        array_dict[f'{tree}/{var}'] = root_file[path_to_var+"/"+var].array(library="np")
print(f'Variables in the TTrees: {array_dict.keys()}') 

In [ ]:
df = pd.DataFrame({'V_raw':array_dict['iv_trim/trim'], 'I_raw':array_dict['iv_trim/current']})
df = pd.DataFrame({'V_raw':array_dict['iv_trim/trim'], 'I_raw':np.flip(array_dict['iv_trim/current'])*(-1)})
df = pd.DataFrame({'V_raw':array_dict['bias/bias_dac'], 'I_raw':array_dict['bias/current']*(-1)})



# for i in range(len(df) - 2):
#     if all(df.iloc[i:i+3]['I_raw'] > 0):
#         df = df.iloc[i:]
#         break

if (df['I_raw'].isna().all()):
    print('ERROR: Current is all NaN')
    print(df)
elif (df['I_raw'].isna().sum() >= 10):
    print('ERROR: More than 10 NaN value for current')
    print(df)
elif (df['I_raw'].isna().sum() > 0) and (df['I_raw'].isna().sum() < 10):
    print('Warning: some NaN value for current, less than 10 NaN. Let continue the analysis')

if ((df['I_raw'] < 0).all()) :
    print('ERROR: Current is negative')
    print(df)

# if (df['I'] < 100).all():
#     print('ERROR: Too low current')
#     print(df)

if (len(df['I_raw']) < 20):
    print('ERROR: Few points, less than 20)')
    print(df)

Plot of the reverse IV curve + a savgol filter:

In [ ]:
# First filter on current
savgol_window = (len(df['V_raw'])) // 15  # 5 seems okay
if (savgol_window < 4) and (savgol_window > 0):
    savgol_window = 4
if (savgol_window == 0):
    savgol_window = 2
df['I_sav'] = savgol_filter(df['I_raw'],savgol_window,1) 

# Create a new dataframe for plotting
df_plot = df.melt(id_vars='V_raw', value_vars=['I_raw', 'I_sav'], var_name='Type', value_name='Current')

fig = px.scatter(df_plot, x='V_raw', y='Current', color='Type', title='REV IV - ENDPOINT:' + str(endpoint) + ' APA:' + str(apa) + ' AFE:' + str(afe) + ' CH:' + str(chn))
fig.update_traces(marker=dict(size=5))
fig.update_layout(xaxis_title='Voltage [DAC]', yaxis_title='Current',template='presentation')
fig.show()

## MATPLOTLIB PLOT ##
# fig, ax = plt.subplots()
# fig.suptitle('REV IV - ENDPOINT:' + str(endpoint) + ' APA:' + str(apa) + ' AFE:' + str(afe) + ' CH:' + str(chn))
# ax.set_xlabel("Trim (DAC)")
# ax.set_ylabel("Current") # unit of measure ?
# # ax.set_yscale('log')
# ax.scatter(df['V_raw'], df['I_raw'], marker='o',s=5, color='blue', label='Acquired IV curve')

# # First filter on current
# savgol_window = (len(df['V_raw'])) // 15  # 5 seems okay
# if (savgol_window < 4) and (savgol_window > 0):
#     savgol_window = 4
# if (savgol_window == 0):
#     savgol_window = 2
# df['I_savgol'] = savgol_filter(df['I_raw'],savgol_window,1) 
# ax.scatter(df['V_raw'], df['I_savgol'], marker='o',s=5, color='deepskyblue', label='Savgol filtered IV curve')

# ax.legend()

Plot of the first normalized derivative (1/I * dI/dV) + second savgol filter on it:

In [ ]:
df['1der_I_raw'] = [i / j for i, j in zip(derivative_anna(df['V_raw'], df['I_raw']), df['I_raw'])]
df['1der_I_sav'] = [i / j for i, j in zip(derivative_anna(df['V_raw'], df['I_sav']), df['I_sav'])]

# Second filter 
df['1der_I_2sav'] = savgol_filter(df['1der_I_sav'],savgol_window*3,2)
peak_index = df['2der_I_sav'].idxmax()
half_point_range = (len(df['V_raw'])) // 12  # 6 seems okay+
if half_point_range < 5: half_point_range = 5
min_index = peak_index - half_point_range
if min_index < 0: min_index = 0
max_index = peak_index + half_point_range 
if max_index < 0: max_index = len(df['V_raw'])-1

df_fit = df.loc[min_index:max_index]
poly2_coeff, poly2_cov = curve_fit(parabolic_function, df_fit['V_raw'].loc[min_index:max_index], df_fit['2der_I_sav'].loc[min_index:max_index])
# poly2_coeff = np.polyfit(df_fit['V_raw'], df_fit['2der_I_sav'], 2)
x_poly2 = np.linspace(df_fit.at[df_fit.index[0],'V_raw'], df_fit.at[df_fit.index[-1],'V_raw'], 100) 
y_poly2 = y_values = np.polyval(poly2_coeff, x_poly2)
print(poly2_coeff)
Vbd = x_poly2[np.argmax(y_poly2)]
print('Vbd = ' + str(Vbd))
print('Vbd = ' + str(Vbd*0.04))

if df['1der_I_2sav'].isna().all() :
    print('ERROR: stop and check you data')
    print(df)
else:
    # Create the plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['V_raw'], y=df['1der_I_raw'],mode='markers',name='1der_I_raw'))
    fig.add_trace(go.Scatter(x=df['V_raw'], y=df['1der_I_sav'],mode='markers',name='1der_I_sav'))
    fig.add_trace(go.Scatter(x=df['V_raw'], y=df['2der_I_sav'],mode='markers',name='1der_I_2sav'))
    fig.add_trace(go.Scatter(x=[df.loc[peak_index, 'V_raw']],y=[df.loc[peak_index, '1der_I_2sav']],
                                mode='markers',marker=dict(color='yellow'),name='Maximum point'))
    fig.add_trace(go.Scatter(x=df_fit['V_raw'],y=df_fit['2der_I_sav'],
                                mode='markers',marker=dict(color='red'),name='Selected points'))
    fig.add_trace(go.Scatter(x=x_poly2, y=y_poly2,mode='lines',marker=dict(color='green'),name='FIT'))

    fig.add_vline(x=Vbd, line_dash="dash", line_color="green",name='Vbd')
    fig.update_traces(marker=dict(size=5))
    fig.update_layout(yaxis_title='Normalized Derivative',template='presentation')
    fig.show()

Search for the maximum of first derivative (filtered):

In [ ]:
## MATPLOTLIB PLOT ##
fig_der, ax_twin = plt.subplots()
ax_twin.set_ylabel('Normalized Derivative')
df['1der_I_raw'] = [i / j for i, j in zip(derivative_anna(df['V_raw'], df['I_raw']), df['I_raw'])]
#df['der_norm'] = np.gradient(df['I']) /df['I']
ax_twin.scatter(df["V_raw"],df['1der_I_raw'], marker='o', s=5, color='purple', label='Derivative of acquired data (no filter)')

df['1der_I_sav'] = [i / j for i, j in zip(derivative_anna(df['V_raw'], df['I_sav']), df['I_sav'])]
ax_twin.scatter(df['V_raw'],df['1der_I_sav'], marker='o', s=5, color='fuchsia', label='Derivative of filtered data')

# Second filter 
df['2der_I_sav'] = savgol_filter(df['1der_I_sav'],savgol_window*3,2)

if df['2der_I_sav'].isna().all() :
    print('ERROR: stop and check you data')
    print(df)
else:
    ax_twin.scatter(df["V_raw"],df['2der_I_sav'], marker='o', s=5, color='orange', label='Derivative of filtered data + Savgol filter')
    #ax_twin.set_ylim([df['der_I_savgol_2'].min()*0.3, df['der_I_savgol_2'].max()*2])

ax_twin.legend()

In [ ]:
# First method: find just the maximum of filtered derivativa
peak_index = df['2der_I_sav'].idxmax()
#ax_twin.scatter(df.loc[peak_index, 'V'],df.loc[peak_index, 'der_I_savgol_2'], marker='o', s=25, color='yellow', label='Maximum point')

# peak_index= (df.iloc[7:-7]['2der_I_sav']).idxmax()
ax_twin.scatter(df.loc[peak_index, 'V_raw'],df.loc[peak_index, '2der_I_sav'], marker='o', s=25, color='yellow', label='Maximum point')


if (peak_index < 5) or (df.loc[peak_index, 'I_raw']<1) :
    print('Error: stop and check data')

# Second method: do a nth polynomial fit and search for its maximum
#degree = 6
#polyfit_par, polyfit_coef = Polynomial.fit(df['V'], df['der_I_savgol_2'], deg = degree, full = True) 
#df['poly_fit'] = [Polynomial(polyfit_par.convert().coef)(i) for i in df['V']]
#peak_index = np.argmax(df['poly_fit']) 
#ax_twin.plot(df['V'][peak_index-40:peak_index+40], df['poly_fit'][peak_index-40:peak_index+40], color='gold', label = 'Polyfit ('+str(degree)+'deg)')
#ax_twin.legend()
fig_der

Select few data around the maximum 

In [ ]:
half_point_range = (len(df['V_raw'])) // 12  # 6 seems okay
if half_point_range < 5:
    half_point_range = 5
min_index = peak_index - half_point_range
if min_index < 0:
    min_index = 0
max_index = peak_index + half_point_range 
if max_index < 0:
    max_index = len(df['V_raw'])-1
df_fit = df.loc[min_index:max_index]

ax_twin.scatter(df_fit['V_raw'],df_fit['2der_I_sav'], marker='o', s=15, color='red', label='Selected points for fit')

ax_twin.legend()
fig_der

Do a parabolic fit (2nd degree polynomial) of the selected data:

In [ ]:
#poly2_par, poly2_coeff = Polynomial.fit(df_fit['V'], df_fit['der_I_savgol_2'], deg = 2, full = True)  
poly2_coeff = np.polyfit(df_fit['V_raw'], df_fit['2der_I_sav'], 2)

x_poly2 = np.linspace(df_fit.at[df_fit.index[0],'V_raw'], df_fit.at[df_fit.index[-1],'V_raw'], 100) 

#y_poly2 = [Polynomial(poly2_par.convert().coef)(i) for i in x_poly2]
y_poly2 = y_values = np.polyval(poly2_coeff, x_poly2)

ax_twin.plot(x_poly2, y_poly2, color='green', label = '2nd deg fit')

ax_twin.set_ylim([df_fit['2der_I_sav'].min()*0.3, df_fit['2der_I_sav'].max()*2])

ax_twin.legend()
fig_der

Search for the breakdown voltage, as the maximum of the parabola:

In [ ]:
Vbd = x_poly2[np.argmax(y_poly2)]
ax_twin.axvline(x=Vbd, color='lime' ,linestyle='--', label=r'$V_{bd} \rightarrow $'+'trim= '+f'{Vbd:.0f} (DAC)')   
print('Vbd = ' + str(Vbd))
#ax_twin.set_xlim([V_bd-50,V_bd+50])
ax_twin.legend()

fig_der